# Colormaps

This Jupyter Notebook describes the rationale for developing the `ehtplot.color` submodule in `ehtplot`.  Before we get into the actual content, let's customize Jupyter so this notebook is easier to read and use.

In [1]:
from IPython.core.display import display, HTML, Javascript 

# Display text with 16px fonts
display(HTML("<style>.rendered_html { font-size: 16px; }</style>"))

# Disable cell scrolling
display(Javascript("IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }"))

# Enable autoreload for easy module developement
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

## 0. Contents:
<a id="sec0"></a>

- [0. Contents](#sec0)
- [1. Background](#sec1)
- [2. Basic Requirements](#sec2)
- [3. Color Models and Color Spaces](#sec3)
- [4. Color Appearance Parameters](#sec4)
- [5. General Guidance](#sec5)
- [6. Colormap Inspectation](#sec6)

## 1. Background
<a id="sec1"></a>

The [Event Horizon Telescope (EHT)](https://eventhorizontelescope.org/) is a Very-long-baseline interferometry (VLBI) experiment aiming at capture of the first pictures of black holes.  While an image worth a thousand words, the interpretation of an image is subjective.  The presentation of an image can strongly affect how human eyes identify features.  This is especially true for two-dimensional intensity maps, where the value of each pixel is represented by a color.  A poorly chosen colormap between values and colors can fool the human eyes to, e.g., pick out non-existing features, or to hide important features.

In order to present the resulting images of the EHT as accurate as possible, we take into account how human eyes work and provide a few guidance on the colormap usages in different situations.

## 2. Basic Requirements
<a id="sec2"></a>

Without putting in too much details, a good colormap for scientific representation of a scalar image should:

1. give readers a correct sense on the scalar values (intensities) in the image;

2. not artificially show features that do not exist;

3. not intentionally hide features that exist;

4. be effective in communicating scientific messages;

5. be device-independent.

We summerize some works on human visual perceptual and standards created by the [International Commission on Illumination (ICE)](https://en.wikipedia.org/wiki/International_Commission_on_Illumination) below.  We will then use these basic knowledge to derive general guidance on deriving and using colormaps.

## 3. Color Models and Color Spaces
<a id="sec3"></a>

The "raw" [RGB](https://en.wikipedia.org/wiki/RGB_color_model) and [CMYK](https://en.wikipedia.org/wiki/CMYK_color_model) [color models](https://en.wikipedia.org/wiki/Color_model) are device-dependent.  In order to describe device-independent color, companies create "standardized" [color spaces](https://en.wikipedia.org/wiki/Color_space) such as [sRGB](https://en.wikipedia.org/wiki/SRGB) (standard RGB by HP and Microsoft) and [Adobe RGB](https://en.wikipedia.org/wiki/Adobe_RGB_color_space). A [color profile](https://en.wikipedia.org/wiki/ICC_profile), either characterizes a device or a color space, can then be used to accurately map the color in an image to a device.

The [CIELAB](https://en.wikipedia.org/wiki/CIELAB_color_space) (also CIE L\*a\*b\*, hereafter Lab) color space, introduced by the CIE in 1976, was the first attempt to take into account the the perceptual aspects of human vision.  Here, L\* is the lightness; a\* and b\* are the green-red and blue-yellow color components.  The [CIECAM02](https://en.wikipedia.org/wiki/CIECAM02) [color appearance model (CAM)](https://en.wikipedia.org/wiki/Color_appearance_model) published by the CIE in 2002 defines six color appearances: brightness Q, lightness J, colorfulness M, chroma C, saturation s, and hue h.  And the [iCAM06](https://en.wikipedia.org/wiki/Color_appearance_model#iCAM06) model is capable of handling spatial appearance phenomena such as contrast.

In 2006, [Luo et al.](https://www.researchgate.net/publication/229879898_Uniform_colour_spaces_based_on_CIECAMO2_colour_appearance_model) introduced three modifications to CIECAM02 to make it more uniform.  The CAM02-UCS, stands for "Uniform Colour Space", is not popular in computer graphics.  Its three coordinates are usually denoted by J', a', and b'.  And its cylindrical coordinates are J', C', and h'.

It is useful to transform the Cartesian Lab and CIECAM02 color spaces to the cylindrical [CIELCh](https://en.wikipedia.org/wiki/CIELAB_color_space#Cylindrical_representation:_CIELCh_or_CIEHLC) (hereafter LCh) and CIEJCh (hereafter JCh) color spaces which has coordinates L\*, J\*, C\*, and h. The lightness coordinates L\* and J\* are identical to Lab and Jab.  The chroma (relative saturation) C\* and hue h (in degree h°) are simply `C* = sqrt(a*^2 + b*^2)` and `h = atan2(b*, a*)` according to Redness-Greenness a and Yellowness-Blueness b in their own coordinates.

## 4. Color Appearance Parameters
<a id="sec4"></a>

We ignore spatial appearance phenomena and consider only the the six color appearance parameters defined by CIECAM02.  They can be grouped into three classes, correspond to the three coordinates of LCh:

- [Brightness](https://en.wikipedia.org/wiki/Brightness) and   [lightness](https://en.wikipedia.org/wiki/Lightness): they are the   extrinsic and intrinsic "tones" or "values", respectively.

- [Colorfulness](https://en.wikipedia.org/wiki/Colorfulness), [chroma](https://en.wikipedia.org/wiki/Colorfulness#Chroma_in_CIE_1976_L*a*b*_and_L*u*v*_color_spaces), and [saturation](https://en.wikipedia.org/wiki/Colorfulness#Saturation): colorfulness and chroma are, roughly speaking, the extrinsic and intrinsic difference between a color and grey of an object, respectively.  Saturation is the colorfulness of a color relative to its own brightness.

- [Hue](https://en.wikipedia.org/wiki/Hue): "the degree to which a stimulus can be described as similar to or different from stimuli that are described as red, green, blue, and yellow."  [Q: are the CIELCh hue and (s)RGB hue `h_rgb = atan2(sqrt(3) * (G-B), 2*R - G - B)` equivalent?]

It is important to note that the above terms can be defined very differently in different context.  For example, RGB brightness is defined as `(R + G + B) / 3`, which is completely different than CIECAM02 brightness.

## 5. General Guidance
<a id="sec5"></a>

In response to the above requirements, at minimal,

- we need to use *perceptually uniform colormaps*

so that the lightness of the color in an image is a fair representation of its scalar values.  While visual perception is a complex science, the CAM02-UCS lightnesses J' should serve us as a good approximation for generating perceptually uniform colormaps.  In fact, linearity in J' is used as the working definition of Perceptually Uniform Sequential colormaps by [matplotlib](https://matplotlib.org/users/colormaps.html).

Since chrominance is a two-dimensional quantity, we can use the different dimensions for different purposes.  Being independent of lightness and chroma and easily recognized,

- *hue h' can encode an additional physical quantity* in an image (when used in this way, the change of hue should be linearly proportional to the quantity);

- *hue h' is also ideal in making an image more attractive* without interferencing the representation of pixel values.

The other dimension chroma is less recognizable and should not be used to encode physical information.  Since sRGB is only a subset of the Lab colorspace, there are human regonizable color that are not displayable.  In order to accurately represent the physical quantities,

- if a color is not displayable, one should preserve its lightness J' and hue h', and adjust its chroma C'.

- *saturation s' is also useful to place focus of an image* without affecting the representation of the physical quantities.

Given that human eyes are less sensitive to color in low light,

- if color is used to encode an additional physical quantity in a figure, the *colormap may start (or end) at non-zero lightness*.

Finally, sRGB is often the default image color space for modern softwares.  That is, if we save an image without a color profile, it is often interpreted as in sRGB with [gamma](https://en.wikipedia.org/wiki/Gamma_correction) 2.2.  Therefore, we should

- convert colormaps to sRGB with gamma 2.2, and then save the resulting images without any color profile.

## 6. Colormap Inspectation
<a id="sec6"></a>

With all the above discussion, we are finally at a point to look into colormaps.  Let's first load up matplotlib and find out what colormaps are available:

In [2]:
import matplotlib.pyplot as plt

colormaps = plt.colormaps()
for cm in colormaps:
    print(cm, end=', ')
print("{} colormaps in total".format(len(colormaps)))

Accent, Accent_r, Blues, Blues_r, BrBG, BrBG_r, BuGn, BuGn_r, BuPu, BuPu_r, CMRmap, CMRmap_r, Dark2, Dark2_r, GnBu, GnBu_r, Greens, Greens_r, Greys, Greys_r, OrRd, OrRd_r, Oranges, Oranges_r, PRGn, PRGn_r, Paired, Paired_r, Pastel1, Pastel1_r, Pastel2, Pastel2_r, PiYG, PiYG_r, PuBu, PuBuGn, PuBuGn_r, PuBu_r, PuOr, PuOr_r, PuRd, PuRd_r, Purples, Purples_r, RdBu, RdBu_r, RdGy, RdGy_r, RdPu, RdPu_r, RdYlBu, RdYlBu_r, RdYlGn, RdYlGn_r, Reds, Reds_r, Set1, Set1_r, Set2, Set2_r, Set3, Set3_r, Spectral, Spectral_r, Wistia, Wistia_r, YlGn, YlGnBu, YlGnBu_r, YlGn_r, YlOrBr, YlOrBr_r, YlOrRd, YlOrRd_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, bone, bone_r, brg, brg_r, bwr, bwr_r, cividis, cividis_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, flag, flag_r, gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnuplot, gnuplot2,

There are 164 built-in colormaps.